<h1 align="center">Analítica de datos para la toma de decisiones empresariales</h1>
<h1 align="center">Serie de Tiempo: Pasajeros de avión</h1>
<h1 align="center">Centro de Educación Continua</h1>
<h1 align="center">EAFIT</h1>
<h1 align="center">2023</h1>
<h1 align="center">MEDELLÍN - COLOMBIA </h1>

*** 
|![Gmail](https://img.shields.io/badge/Gmail-D14836?style=plastic&logo=gmail&logoColor=white)|<carlosalvarezh@gmail.com>|![Outlook](https://img.shields.io/badge/Microsoft_Outlook-0078D4?style=plastic&logo=microsoft-outlook&logoColor=white)|<calvar52@eafit.edu.co>|
|-:|:-|--:|:--|
|[![LinkedIn](https://img.shields.io/badge/linkedin-%230077B5.svg?style=plastic&logo=linkedin&logoColor=white)](https://www.linkedin.com/in/carlosalvarez5/)|[![@alvarezhenao](https://img.shields.io/twitter/url/https/twitter.com/alvarezhenao.svg?style=social&label=Follow%20%40alvarezhenao)](https://twitter.com/alvarezhenao)|[![@carlosalvarezh](https://img.shields.io/badge/github-%23121011.svg?style=plastic&logo=github&logoColor=white)](https://github.com/carlosalvarezh)|[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/carlosalvarezh/Curso_CEC_EAFIT/blob/main/Ejemplos_C02_5_TimeSeriesAnalysis.ipynb)|

<table>
 <tr align=left><td><img align=left src="https://github.com/carlosalvarezh/Curso_CEC_EAFIT/blob/main/images/CCLogoColorPop1.gif?raw=true" width="25">
 <td>Text provided under a Creative Commons Attribution license, CC-BY. All code is made available under the FSF-approved MIT license.(c) Carlos Alberto Alvarez Henao</td>
</table>

***

## Introducción

En el mundo empresarial actual, la capacidad de tomar decisiones informadas y estratégicas es esencial para el éxito. Una valiosa herramienta para lograrlo es el análisis de datos de series temporales. En una amplia variedad de sectores, desde finanzas hasta marketing, las organizaciones recopilan información a lo largo del tiempo, como precios de acciones, consumo de energía y métricas de redes sociales. Este tipo de datos pueden revelar patrones, tendencias estacionales y pronósticos futuros que impulsan la generación de ganancias. Por ejemplo, al comprender cómo varía la demanda de productos a lo largo del año, las empresas pueden planificar estratégicamente promociones para optimizar las ventas. En este contexto, exploraremos un enfoque práctico utilizando el método ARIMA para analizar públicamente disponibles datos de pasajeros de aerolíneas en Python. A través de este ejemplo, aprenderemos a descomponer tendencias, identificar patrones y pronosticar eventos futuros, fortaleciendo así nuestras habilidades en la toma de decisiones basadas en datos.

## Carga de datos

Empecemos el ejercicio con la importación de las bibliotecas a trabajar y la lectura de los datos

In [ ]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns 
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose

plt.rcParams.update({'figure.figsize': (6, 3), 'figure.dpi': 120})

Los datos de una serie temporal normalmente se almacenan en archivos `.csv` u otros formatos de hoja de cálculo y contienen dos columnas: *la fecha* y *el valor medido*.

Usaremos `read_csv()` del paquete `pandas` para leer el conjunto de datos de series temporales como un `dataset` de `pandas`. Agregar el argumento `parse_dates=['Month']` hará que la columna de fecha se analice como un campo de fecha.

In [ ]:
df = pd.read_csv("Data/AirPassengers.csv")

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

Alternativamente, puedes importarlo como una serie de pandas con la fecha como índice. Sólo necesita especificar el argumento `index_col` en `pd.read_csv()` para hacerlo:

```python
serie = pd.read_csv("Data/AirPassengers.csv", parse_dates=['Month'], index_col='Month')
```

Se deja como texto para evitar ponerlo en una línea celda ejecutable, ya que puede alterar los resultados del código de este documento.

 ## Análisis Exploratorio

Vamos a continuar trabajando con el DataFrame `df` creado. Podemos observar que los datos contienen una columna etiquetada como "*Month*" que contiene fechas. En esa columna, las fechas están formateadas como año-mes. También vemos que los datos comienzan en enero de $1949$ y van hasta diciembre de $1960$.

La segunda columna está etiquetada como "*#Passangers*" y contiene el número de pasajeros para el año-mes.

Veamos cuántos elementos contiene el dataset y cuántos son no-nulos:

In [ ]:
null_data = df.isnull()
null_data

Si el dataset cuenta con valores faltantes y quisiéramos determinar en qué fila se encuentran, podemos ejecutar este código

In [ ]:
rows_with_missing_indices = df[df.isna().any(axis=1)].index
print(rows_with_missing_indices)

O si lo que necesitamos es saber la posición exacta del elemento nulo, podemos emplear este código:

In [ ]:
# Obtener la posición exacta de los valores nulos
rows, cols = np.where(df.isna())

# Crear una lista de tuplas con las posiciones
null_positions = list(zip(rows, cols))

print(null_positions)

En donde el primer elemento de la tupla corresponde a la fila y el segundo, a la columna.

Si al cargar los datos hemos obviado el argumento `parse_dates=['Month']`, que transforma la columna *Month* en fechas, lo podremos hacer con el método [`to_datetime()`](https://pandas.pydata.org/docs/reference/api/pandas.to_datetime.html)  de `Pandas`. Adicionalmente empleamos el argumento `format='%Y-%m'`, para indicar el formato de fecha deseado:

In [ ]:
df['Month'] = pd.to_datetime(df['Month'], format='%Y-%m')
df.head()

In [ ]:
df.info()

Este proceso automáticamente inserta el primer día de cada mes, lo cual es básicamente un valor ficticio dado que no tenemos datos diarios de pasajeros.

Observe también que el tipo de dato cambio de `object` a `datetime64[ns]`

Grafiquemos los datos del DataFrame utilizando el comando [`lineplot()`](https://seaborn.pydata.org/generated/seaborn.lineplot.html) de `seaborn`, que nos permite visualziar un gráfico de líneas

In [ ]:
sns.lineplot(data = df, x="Month", y = "#Passengers")
plt.ylabel('Number of Passengers')

Dado que es una serie temporal mensual y sigue un cierto patrón repetitivo cada año, puedes trazar cada año como una línea separada en el mismo gráfico. Esto le permite comparar los patrones anuales uno al lado del otro.

In [ ]:
df1 = df.copy()

In [ ]:
import matplotlib as mpl

# Prepare data
df1['year'] = [d.year for d in df1["Month"]]
df1['Month'] = [d.strftime('%b') for d in df1["Month"]]
years = df1['year'].unique()

# Prep Colors
np.random.seed(100)
mycolors = np.random.choice(list(mpl.colors.XKCD_COLORS.keys()), len(years), replace=False)

# Draw Plot
plt.figure(figsize=(10,6), dpi= 80)
for i, y in enumerate(years):
    if i > 0:        
        plt.plot('Month', '#Passengers', data=df1.loc[df1.year==y, :], color=mycolors[i], label=y)
        plt.text(df1.loc[df1.year==y, :].shape[0]-.9, df1.loc[df1.year==y, '#Passengers'][-1:].values[0], y, fontsize=12, color=mycolors[i])

# Decoration
xmin = -0.3
xmax = df1['Month'].nunique()
ymin = df1['#Passengers'].min()
ymax = df1['#Passengers'].max()

plt.gca().set(xlim=(xmin, xmax), ylim=(ymin, ymax), ylabel='#Passengers', xlabel='Month')
plt.yticks(fontsize=12, alpha=.7)
plt.title("#Passengers Time Series", fontsize=20)
plt.show()


De la gráfica se observa que, a partir de $1954$, se presenta una leve caída en el número de pasajeros el mes de enero, y luego empieza a crecer sostenidamente hasta su máximo en julio, para irse en tendencia decreciente hasta noviembre y volver a subir un poco en diciembre. Este patrón se evidencia que se repite año a año.

***Diagrama de caja de distribución mensual (estacional) y anual (tendencia)***

Ahora, agrupemos los datos en intervalos estacionales y ver cómo se distribuyen los valores dentro de un año o mes determinado y cómo se comparan a lo largo del tiempo.

In [ ]:
# Draw Plot
fig, axes = plt.subplots(1, 2, figsize=(20,7), dpi= 80)
sns.boxplot(x='year', y='#Passengers', data=df1, ax=axes[0])
sns.boxplot(x='Month', y='#Passengers', data=df1.loc[~df1.year.isin([1991, 2008]), :])

# Set Title
axes[0].set_title('Year-wise Box Plot\n(The Trend)', fontsize=18); 
axes[1].set_title('Month-wise Box Plot\n(The Seasonality)', fontsize=18)
plt.show()

Los diagramas de caja hacen evidentes las distribuciones anuales y mensuales. Además, viendo en el diagrama de caja mensual, los meses de *junio*, *julio* y *agosto* claramente tienen mayores ventas de pasajes, lo que puede atribuirse a la temporada de vacaciones de verano en el hemisferio norte.

Hasta ahora, hemos visto las similitudes para identificar el patrón. Ahora bien, ¿cómo descubrir cualquier desviación del patrón habitual?

## Análisis de Estacionariedad

La [estacionariedad](https://en.wikipedia.org/wiki/Stationary_process) es esencial en el análisis de series temporales, indicando que los cambios en los datos a lo largo del tiempo son constantes y no muestran tendencias ni patrones estacionales. Esto facilita la modelización y es un supuesto en muchos métodos de pronóstico. Utilizaremos la prueba [*Dickey Fuller*](https://en.wikipedia.org/wiki/Dickey%E2%80%93Fuller_test) para verificar la estacionariedad en nuestros datos, permitiéndonos aceptar o rechazar la hipótesis de no estacionariedad. Si rechazamos la hipótesis nula, confirmamos la existencia de estacionariedad, lo que es crucial para evaluar cómo los valores actuales se relacionan con los pasados en el conjunto de datos.

### Media Móvil

La [media móvil](https://en.wikipedia.org/wiki/Moving_average) se calcula como una técnica para suavizar las fluctuaciones en los datos y resaltar las tendencias subyacentes a lo largo del tiempo. Se utiliza en análisis de series temporales para reducir el ruido o variabilidad aleatoria en los datos, lo que facilita la identificación de patrones y tendencias más claras.

La idea detrás de la media móvil es tomar un conjunto de valores consecutivos de una serie temporal y calcular su promedio. A medida que avanzamos en el tiempo, se va ajustando la ventana de valores que se promedian, lo que suaviza las fluctuaciones a corto plazo y revela las tendencias a largo plazo.

La media móvil es especialmente útil para detectar patrones y tendencias en datos que pueden tener variaciones estacionales, cíclicas o aleatorias. Puede ser una herramienta efectiva para la exploración preliminar de datos y proporcionar una base para el análisis más profundo en el campo de la estadística y el análisis de series temporales.

Para el ejemplo, calculemos una media móvil de siete meses:

In [ ]:
df.set_index('Month', inplace=True)
df

In [ ]:
rolling_mean = df.rolling(7).mean()
rolling_std = df.rolling(7).std()

A continuación, vamos a superponer nuestra serie temporal con la media móvil de 7 meses y la desviación estándar móvil de siete meses. En primer lugar, creemos una gráfica utilizando Matplotlib de nuestra serie temporal, junto con la media móvil a 7 meses y su desviación estándar:

In [ ]:
plt.plot(df, color="blue",label="Original Passenger Data")
plt.plot(rolling_mean, color="red", label="Rolling Mean Passenger Number")
plt.plot(rolling_std, color="black", label = "Rolling Standard Deviation in Passenger Number")

plt.title("Passenger Time Series, Rolling Mean, Standard Deviation")
plt.legend(loc="best")

### Pruebas estadísticas

A continuación, importemos la prueba aumentada [Dickey-Fuller](https://en.wikipedia.org/wiki/Dickey%E2%80%93Fuller_test) del paquete statsmodels. La documentación de la prueba se puede encontrar [aquí](https://www.statsmodels.org/dev/generated/statsmodels.tsa.stattools.adfuller.html). Luego, pasemos nuestro *DataFrame* al método `adfuller`. Aquí, especificamos el parámetro `autolag` como "`AIC`", lo que significa que se elige el rezago para minimizar el criterio de información. Finalmente, almacenemos nuestros resultados en un *DataFrame* y lo mostremos:

In [ ]:
adft = adfuller(df,autolag="AIC")

adftValues = [adft[0],adft[1],adft[2],adft[3], adft[4]['1%'], adft[4]['5%'], adft[4]['10%']]

metrics = ["Test Statistics","p-value","No. of lags used","Number of observations used",
           "critical value (1%)", "critical value (5%)", "critical value (10%)"]

output_df = pd.DataFrame({"Values": adftValues, "Metric": metrics})
print(output_df)

Podemos observar que nuestros datos no son estacionarios debido a que nuestro valor $p> 5\%$ y el estadístico de prueba es mayor al valor crítico. También podemos llegar a estas conclusiones al inspeccionar los datos, ya que podemos ver claramente una tendencia creciente en el número de pasajeros.

### Autocorrelación

Verificar la [autocorrelación](https://en.wikipedia.org/wiki/Autocorrelation) en los datos de series temporales en Python es otra parte importante del proceso analítico. Esto es una medida de cómo está correlacionada la serie temporal en un punto dado en el tiempo con los valores pasados, lo que tiene implicaciones significativas en diversas industrias. Por ejemplo, si nuestros datos de pasajeros tienen una fuerte autocorrelación, podemos asumir que un alto número de pasajeros hoy sugiere una alta probabilidad de que también sean altos mañana.

Pandas tiene un método de autocorrelación que podemos utilizar para calcular la autocorrelación en nuestros datos de pasajeros. Hagámoslo para un rezago de uno, tres, seis y nueve meses:

In [ ]:
autocorrelation_lag1 = df['#Passengers'].autocorr(lag=1)
print("One Month Lag: ", autocorrelation_lag1)

autocorrelation_lag3 = df['#Passengers'].autocorr(lag=3)
print("Three Month Lag: ", autocorrelation_lag3)

autocorrelation_lag6 = df['#Passengers'].autocorr(lag=6)
print("Six Month Lag: ", autocorrelation_lag6)

autocorrelation_lag9 = df['#Passengers'].autocorr(lag=9)
print("Nine Month Lag: ", autocorrelation_lag9)

In [ ]:
pd.plotting.autocorrelation_plot(df['#Passengers'])
plt.show()

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf
plot_acf(df['#Passengers'], lags=30)

Vemos que, incluso con un retraso de nueve meses, los datos presentan una autocorrelación alta. Esto es una evidencia adicional de las tendencias a corto y largo plazo en los datos.

### Descomposición

La [descomposición de tendencias](https://en.wikipedia.org/wiki/Decomposition_of_time_series) es una técnica utilizada en el análisis de series temporales para separar una serie temporal en sus componentes fundamentales, con el objetivo de comprender mejor la estructura subyacente de los datos. Esta técnica se utiliza para identificar y aislar las tendencias, las estacionalidades y los componentes residuales en una serie de tiempo.

A continuación, se explican los tres componentes principales que se buscan en la descomposición de tendencias:

1. ***Tendencia:*** La tendencia es la componente de una serie temporal que muestra la dirección general en la que los datos están cambiando a lo largo del tiempo. Puede ser una tendencia ascendente (crecimiento), descendente (decrecimiento) o lateral (sin cambio significativo). La descomposición de tendencias busca separar esta componente para identificar patrones a largo plazo en los datos.
<p>&nbsp;</p>

2. ***Estacionalidad:*** La estacionalidad es la componente que representa patrones recurrentes y predecibles que ocurren en la serie de tiempo a intervalos regulares. Por ejemplo, las ventas minoristas pueden experimentar un aumento estacional durante las vacaciones cada año. La descomposición de tendencias identifica estos patrones estacionales y los separa de la tendencia general.
<p>&nbsp;</p>

3. ***Residuos:*** Los residuos son la parte de la serie de tiempo que no se puede explicar mediante la tendencia y la estacionalidad. Representan la variabilidad no sistemática o aleatoria en los datos. Los residuos son importantes porque pueden contener información sobre eventos impredecibles o cambios no modelados en la serie temporal.

La descomposición de tendencias se puede realizar utilizando diversos métodos, siendo dos de los más comunes:

- ***Método aditivo:*** En este enfoque, se considera que la serie temporal es la suma de la tendencia, la estacionalidad y los residuos. La descomposición implica restar la tendencia y la estacionalidad a los datos originales para obtener los residuos.
<p>&nbsp;</p>

- ***Método multiplicativo:*** En este enfoque, se considera que la serie temporal es el producto de la tendencia, la estacionalidad y los residuos. La descomposición implica dividir los datos originales por la tendencia y la estacionalidad para obtener los residuos.

Una vez que se ha realizado la descomposición de tendencias, los analistas de series temporales pueden examinar cada componente por separado para comprender mejor las tendencias a largo plazo, los patrones estacionales y la variabilidad no sistemática en los datos. Esto es útil para la predicción y el modelado de series temporales, ya que permite abordar cada componente de manera individual y aplicar técnicas específicas a cada uno de ellos.




Para continuar, importemos `seasonal_decompose` del paquete `statsmodels`. Pasemos nuestro *DataFrame* al método `season_decompose` y grafiquemos el resultado:

In [ ]:
decompose_add = seasonal_decompose(df['#Passengers'],model='additive', period=7)
plt.rcParams.update({'figure.figsize': (10,5)})
decompose_add.plot().suptitle('Additive Decompose', fontsize=10)
plt.show()

El código que se acaba de ejecutar se utiliza para realizar primero la descomposición estacional aditivaen una serie de tiempo contenida en un DataFrame de Pandas llamado `df`. Aquí está la explicación de cada parte del código:

- `seasonal_decompose`: Este es un método que se utiliza para realizar la descomposición estacional de una serie de tiempo. Es parte de la biblioteca de Python llamada `statsmodels`. El método `seasonal_decompose` toma como entrada la serie de tiempo que se desea descomponer y varios parámetros para controlar el proceso de descomposición.
<p>&nbsp;</p>

- `df['#Passengers']`: Aquí se accede a la columna llamada `#Passengers` en el DataFrame `df`. Se asume que esta columna contiene la serie de tiempo que deseas descomponer.
<p>&nbsp;</p>

- `model='additive'`: Este parámetro especifica el tipo de modelo que se utilizará para la descomposición estacional. En este caso, se está utilizando el modelo aditivo, lo que significa que se asume que la serie de tiempo es la suma de la tendencia, la estacionalidad y los residuos.
<p>&nbsp;</p>

- `period=7`: Este parámetro indica el período de estacionalidad en la serie de tiempo. En este caso, se establece en 7, lo que sugiere que hay un patrón estacional que se repite cada 7 observaciones. Esto podría significar que estás trabajando con datos diarios y esperas una estacionalidad semanal.

Después de ejecutar este código, la variable `decompose` contendrá un objeto que contiene la descomposición de la serie de tiempo en sus componentes principales, que son:

- `decompose.trend`: La componente de tendencia, que representa la dirección general de los datos a lo largo del tiempo.
<p>&nbsp;</p>

- `decompose.seasonal`: La componente estacional, que muestra los patrones que se repiten a lo largo de un período determinado (en este caso, 7).
<p>&nbsp;</p>

- `decompose.resid`: Los residuos, que representan la variabilidad no sistemática o aleatoria en los datos.

Se puede acceder a cada uno de estos componentes a través del objeto `decompose` para analizarlos o visualizarlos por separado. Esta descomposición puede ser útil para comprender mejor la estructura de la serie de tiempo y facilitar el modelado y la predicción de datos futuros.

La versión de la descomposición multiplicativa se muestra a seguir:

In [ ]:
decompose_mul = seasonal_decompose(df['#Passengers'],model='multiplicative', period=7)
plt.rcParams.update({'figure.figsize': (10,5)})
decompose_mul.plot().suptitle('Multiplicative Decompose', fontsize=10)
plt.show()

### Pronóstico (Forecasting)

El [pronóstico de series temporales](https://en.wikipedia.org/wiki/Time_series) nos permite predecir valores futuros en una serie temporal dada la información actual y pasada. Aquí, utilizaremos el método [ARIMA](https://en.wikipedia.org/wiki/Autoregressive_integrated_moving_average) para pronosticar el número de pasajeros, lo que nos permitirá pronosticar valores futuros en función de una combinación lineal de valores pasados. Utilizaremos el paquete `auto_arima`, lo que nos evitará el proceso de ajuste de hiperparámetros que consume mucho tiempo.

Primero, dividamos nuestros datos para entrenamiento y prueba, y visualicemos la división:

In [ ]:
df['Date'] = df.index
train = df[df['Date'] < pd.to_datetime("1960-08", format='%Y-%m')]
train['train'] = train['#Passengers']
del train['Date']
del train['#Passengers']
test = df[df['Date'] >= pd.to_datetime("1960-08", format='%Y-%m')]
del test['Date']
test['test'] = test['#Passengers']
del test['#Passengers']
plt.plot(train, color = "black")
plt.plot(test, color = "red")
plt.title("Train/Test split for Passenger Data")
plt.ylabel("Passenger Number")
plt.xlabel('Year-Month')
plt.grid(True)
sns.set()
plt.show()

La línea negra corresponde a nuestros datos de entrenamiento (`train`) y la línea roja corresponde a nuestros datos de prueba (`test`).

Importemos `auto_arima` del paquete `pdmarima`, entrenemos nuestro modelo y generemos predicciones.

<div class="alert alert alert-success">
$\color{red}{\textbf{Nota:}}$ Si es la primera vez que se va a ejecutar este paquete en su sistema (computador), debes instalarlo ejecutando la siguiente celda. Una vez instalado, no será necesario volverla a ejecutar, ni tampoco en ningún otro proyecto (notebook) que use este computador:
</div>


In [ ]:
pip install pmdarima

In [ ]:
from pmdarima.arima import auto_arima
model = auto_arima(train, trace=True, error_action='ignore', suppress_warnings=True)
model.fit(train)
forecast = model.predict(n_periods=len(test))
forecast = pd.DataFrame(forecast,index = test.index,columns=['Prediction'])

Ahora, grafiquemos los resultados

In [ ]:
plt.plot(train, label='Train')
plt.plot(test, label='Test')
plt.plot(forecast, label='Prediction')
plt.title('#Passenger Prediction')
plt.xlabel('Date')
plt.ylabel('Actual #Passenger')
plt.legend(loc='upper left', fontsize=8)
plt.show()

Nuestras predicciones se muestran en verde y los valores reales se muestran en naranja.

Finalmente, calculemos el error cuadrático medio (RMSE):

In [ ]:
from math import sqrt
from sklearn.metrics import mean_squared_error
rms = sqrt(mean_squared_error(test,forecast))
print("RMSE: ", rms)

Un valor de *RMSE* (*Root Mean Square Error*) de $64.4816^*$ indica la raíz del promedio de los errores al cuadrado entre los valores pronosticados y los valores reales en un conjunto de datos. Es una medida de la diferencia entre los valores pronosticados y los valores observados en una serie temporal. Cuanto menor sea el valor del RMSE, mejor será el ajuste del modelo a los datos observados.

En este contexto, un *RMSE* de $64.4816^*$ significa que, en promedio, los pronósticos del modelo difieren en alrededor de $64.4816^*$ unidades de la variable objetivo (en este caso, el número de pasajeros) de los valores reales en el conjunto de datos. Es importante interpretar este valor en relación con la escala de la variable y el contexto específico de la aplicación. Un *RMSE* más bajo indica que el modelo está haciendo predicciones más precisas y cercanas a los valores reales.

($^*$Este valor cambia cada vez que hacemos una nueva predicción)

### Análisis

El análisis de datos de series temporales desempeña un papel fundamental. Este análisis es una tarea que enfrentarán casi todos los profesionales en su carrera, y comprender las herramientas y métodos adecuados puede capacitar a los participantes para descubrir tendencias, anticipar eventos y, por ende, influir en la toma de decisiones estratégicas. A través de la comprensión de patrones estacionales mediante conceptos como estacionariedad, autocorrelación y descomposición de tendencias, se puede orientar la planificación de promociones a lo largo del año, mejorando así los resultados financieros de las empresas. Por último, la capacidad de prever eventos futuros mediante pronósticos de series temporales se convierte en una herramienta poderosa para guiar la toma de decisiones empresariales. Estos análisis se vuelven invaluables para cualquier profesional o equipo que busque aportar un valor significativo a sus organizaciones a través de la interpretación y aplicación de datos de series temporales